In [1]:
# -*- coding: utf-8
# Repórter Brasil (http://ruralometro.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Programa para criar base de deputados 2014, com CPF e id da API da Câmara

In [ ]:
import pandas as pd
from pathlib import Path
import requests
import unidecode

Funcao para ler arquivos em diretorio - 2014

In [2]:
def _reader_2014(fname):
    return pd.read_csv(fname, sep=';', header=None, encoding='latin_1', converters={13: lambda x: str(x), 11: lambda x: str(x)})

Função para retirar acentuação

In [2]:
def f(str):
    return (unidecode.unidecode(str))

Carrega os candidatos de 2014

Os arquivos oficiais de candidatos são do TSE (http://agencia.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2014.zip)

Os arquivos TXT foram armazenados numa pasta chamada candidatos_2014. O programa vai todos os arquivos e juntá-los em um

In [4]:
folder = Path("candidatos_2014")
candidatos_2014 = pd.concat([
    _reader_2014(txt)
    for txt in folder.glob("*.txt")
])

In [5]:
# Apaga as colunas que não serão usadas
del candidatos_2014[0]
del candidatos_2014[1]
del candidatos_2014[2]
del candidatos_2014[3]
del candidatos_2014[4]
del candidatos_2014[6]
del candidatos_2014[7]
del candidatos_2014[8]
del candidatos_2014[12]
del candidatos_2014[15]
del candidatos_2014[16]
del candidatos_2014[17]
del candidatos_2014[19]
del candidatos_2014[20]
del candidatos_2014[21]
del candidatos_2014[22]
del candidatos_2014[23]
del candidatos_2014[24]
del candidatos_2014[25]
del candidatos_2014[26]
del candidatos_2014[27]
del candidatos_2014[28]
del candidatos_2014[29]
del candidatos_2014[30]
del candidatos_2014[31]
del candidatos_2014[32]
del candidatos_2014[33]
del candidatos_2014[34]
del candidatos_2014[35]
del candidatos_2014[36]
del candidatos_2014[37]
del candidatos_2014[38]
del candidatos_2014[39]
del candidatos_2014[40]
del candidatos_2014[41]
del candidatos_2014[42]
del candidatos_2014[43]
del candidatos_2014[45]

Dá nomes para as colunas

In [6]:
candidatos_2014.columns = ['uf', 'cargo', 'nome_completo', 'sequencial', 'cpf', 'nome_urna', 'partido_eleicao', 'situacao']

In [7]:
candidatos_2014["nome_completo"] = candidatos_2014["nome_completo"].apply(f)

In [18]:
candidatos_2014.to_csv('resultados/candidatos_2014.csv', index=False)

In [8]:
candidatos_2014.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26245 entries, 0 to 1063
Data columns (total 8 columns):
uf                 26245 non-null object
cargo              26245 non-null object
nome_completo      26245 non-null object
sequencial         26245 non-null object
cpf                26245 non-null object
nome_urna          26245 non-null object
partido_eleicao    26245 non-null object
situacao           26245 non-null object
dtypes: object(8)
memory usage: 1.8+ MB


In [21]:
nome = candidatos_2014[(candidatos_2014['nome_completo'] == 'CELSO ALENCAR RAMOS JACOB') ]
nome

,uf,cargo,nome_completo,sequencial,cpf,nome_urna,partido_eleicao,situacao
1058,RJ,DEPUTADO FEDERAL,CELSO ALENCAR RAMOS JACOB,190000001433,38108216753,CELSO JACOB,PMDB,SUPLENTE


Separa apenas deputados eleitos em 2014

Revogação da liminar que atribuiu efeito suspensivo ao Recurso Extraordinário 847135, proferida na Ação Cautelar 3778, conforme mensagem do STF - o Tribunal procedeu à nova retotalização do resultado das Eleições de 2014. Foram computados como nulos os 16379 votos recebidos por Geraldo Hilario Torres. Com isso, Franklin passa à condição de 1º suplente e Adelmo Leão passa à condição de eleito.

E os deputados que morreram: <br>
João Castelo (PSDB) ->  assumiu Luana Costa (PSB) <br>
Rômulo Gouveia (PSD) -> assumiu Marcondes Gadelha (PSC)

In [20]:
eleitos = candidatos_2014[(candidatos_2014['situacao'] == 'ELEITO POR QP') | 
                          (candidatos_2014['situacao'] == 'ELEITO POR MÉDIA') | 
                          (candidatos_2014['situacao'] == 'ELEITO') |
                          (candidatos_2014['nome_completo'] == 'ADELMO CARNEIRO LEAO') |
                          (candidatos_2014['nome_completo'] == 'LUANA MARIA DA SILVA COSTA')|
                          (candidatos_2014['nome_completo'] == 'MARCONDES IRAN BENEVIDES GADELHA')
                         ]
eleitos_d = eleitos[(eleitos['cargo'] == 'DEPUTADO FEDERAL') & 
                    (eleitos['nome_completo'] != 'FRANKLIN ROBERTO DE LIMA SOUZA') & 
                     (eleitos['nome_completo'] != 'JOAO CASTELO RIBEIRO GONCALVES') & 
                     (eleitos['nome_completo'] != 'ROMULO JOSE DE GOUVEIA') 
                   ]

Alternativa - todos como foram eleitos em 2014

In [ ]:
eleitos_d = candidatos_2014[(candidatos_2014['situacao'] == 'ELEITO POR QP') | 
                          (candidatos_2014['situacao'] == 'ELEITO POR MÉDIA') | 
                          (candidatos_2014['situacao'] == 'ELEITO') 
                         ]

In [21]:
eleitos_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 144 to 1028
Data columns (total 8 columns):
uf                 513 non-null object
cargo              513 non-null object
nome_completo      513 non-null object
sequencial         513 non-null object
cpf                513 non-null object
nome_urna          513 non-null object
partido_eleicao    513 non-null object
situacao           513 non-null object
dtypes: object(8)
memory usage: 36.1+ KB


In [22]:
eleitos_d.to_excel('resultados/eleitos_2014_tse.xlsx',sheet_name='Sheet1')

## Cria dataframe com deputado id, da API da Câmara

Documentação: dadosabertos.camara.leg.br/swagger/api.html

In [23]:
url = 'https://dadosabertos.camara.leg.br/api/v2/deputados'

In [24]:
deputados = []
for pagina in [1, 2, 3, 4, 5, 6, 7, 8]:
    parametros = {'formato': 'json', 'itens': 100, 'pagina': pagina}
    resposta = requests.get(url, parametros)
    for deputado in resposta.json()['dados']:
        dicionario = {"deputado": deputado['nome'], "link_api": deputado['uri']}
        deputados.append(dicionario)

In [25]:
df_deputados_api = pd.DataFrame(deputados)

In [26]:
df_deputados_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 2 columns):
deputado    513 non-null object
link_api    513 non-null object
dtypes: object(2)
memory usage: 8.1+ KB


In [27]:
deputados_dados = []

for num, row in df_deputados_api.iterrows():
    url = (row['link_api'])
    deputado = (row['deputado'])
    #print('deputado: ' + deputado)
    parametros = {'formato': 'json', 'itens': 100}
    resposta = requests.get(url, parametros)
    dicionario = {"deputado_id": str(resposta.json()['dados']['id']),
                      "nome_completo": resposta.json()['dados']['nomeCivil'],
                      "uri": url, 
                      "partido": resposta.json()['dados']['ultimoStatus']['siglaPartido'],
                      "legislatura": resposta.json()['dados']['ultimoStatus']['idLegislatura'],
                      "situacao": resposta.json()['dados']['ultimoStatus']['situacao'],
                      "condicao": resposta.json()['dados']['ultimoStatus']['condicaoEleitoral']}
    deputados_dados.append(dicionario)

In [28]:
deputados_dados_api = pd.DataFrame(deputados_dados)

In [29]:
deputados_dados_api["nome_completo"] = deputados_dados_api["nome_completo"].apply(f)

In [30]:
deputados_dados_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 7 columns):
condicao         513 non-null object
deputado_id      513 non-null object
legislatura      513 non-null int64
nome_completo    513 non-null object
partido          513 non-null object
situacao         513 non-null object
uri              513 non-null object
dtypes: int64(1), object(6)
memory usage: 28.1+ KB


In [34]:
nome = deputados_dados_api[(deputados_dados_api['nome_completo'] == 'MARCONDES IRAN BENEVIDES GADELHA')]
nome

,condicao,deputado_id,legislatura,nome_completo,partido,situacao,uri
326,Efetivado,74470,55,MARCONDES IRAN BENEVIDES GADELHA,PSC,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...


Comparação com os nomes completos das bases

In [31]:
df_eleitos = pd.merge(eleitos_d, deputados_dados_api, left_on='nome_completo', right_on='nome_completo')

In [32]:
df_eleitos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472 entries, 0 to 471
Data columns (total 14 columns):
uf                 472 non-null object
cargo              472 non-null object
nome_completo      472 non-null object
sequencial         472 non-null object
cpf                472 non-null object
nome_urna          472 non-null object
partido_eleicao    472 non-null object
situacao_x         472 non-null object
condicao           472 non-null object
deputado_id        472 non-null object
legislatura        472 non-null int64
partido            472 non-null object
situacao_y         472 non-null object
uri                472 non-null object
dtypes: int64(1), object(13)
memory usage: 55.3+ KB


In [28]:
df_eleitos.to_csv('resultados/dep_eleitos_2014_parcial1.csv', index=False)

In [29]:
df_eleitos.to_excel('resultados/dep_eleitos_2014_parcial1.xlsx',sheet_name='Sheet1')

Comparação mostra 43 nomes diferentes - podem ser nomes diferentes deputados que já deixaram o cargo

outer: usa união de chaves de ambos os quadros, semelhante a uma junção externa completa do SQL; classificar chaves lexicograficamente
indicator: Se True, adiciona uma coluna para saída DataFrame chamado "_merge" com informações sobre a origem de cada linha. Se uma seqüência de caracteres, a coluna com informações sobre a fonte de cada linha será adicionada para saída DataFrame, e a coluna será denominada valor da string. A coluna de informação é de tipo Categorical e assume um valor de "left_only" para observações cuja chave de mesclagem só aparece no DataFrame 'esquerdo', "right_only" para observações cuja chave de mesclagem aparece apenas no DataFrame 'direito' e "ambos" se o A chave de mesclagem da observação é encontrada em ambos.

In [23]:
nomes_naoeleitos = pd.merge(eleitos_d, 
                        deputados_dados_api, 
                        left_on='nome_completo', 
                        right_on='nome_completo',
                        how='outer',
                        indicator=True)

Agrupa apenas pelo lado esquerdo - a lista do TSE

In [24]:
ldf = nomes_naoeleitos.query("_merge == 'left_only'").drop('_merge',axis=1)

In [25]:
ldf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 22 to 508
Data columns (total 14 columns):
uf                 43 non-null object
cargo              43 non-null object
nome_completo      43 non-null object
sequencial         43 non-null object
cpf                43 non-null object
nome_urna          43 non-null object
partido_eleicao    43 non-null object
situacao_x         43 non-null object
condicao           0 non-null object
deputado_id        0 non-null object
legislatura        0 non-null float64
partido            0 non-null object
situacao_y         0 non-null object
uri                0 non-null object
dtypes: float64(1), object(13)
memory usage: 5.0+ KB


In [34]:
ldf.to_csv('resultados/dep_eleitos_2014_parcial2.csv', index=False)
ldf.to_excel('resultados/dep_eleitos_2014_parcial2.xlsx',sheet_name='Sheet1')

Nesse ponto fiz uma checagem manual dos 43 nomes, adicionado a coluna chave que acessa a API de cada deputado

In [26]:
deputados_restantes = pd.read_excel('resultados/dep_eleitos_2014_parcial2.xlsx',sheet_name='Sheet1')

In [27]:
deputados_restantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 22 to 508
Data columns (total 15 columns):
uf                 43 non-null object
cargo              43 non-null object
nome_completo      43 non-null object
sequencial         43 non-null int64
cpf                43 non-null int64
nome_urna          43 non-null object
partido_eleicao    43 non-null object
situacao_x         43 non-null object
condicao           0 non-null float64
deputado_id        0 non-null float64
legislatura        0 non-null float64
partido            0 non-null float64
situacao_y         0 non-null float64
uri                0 non-null float64
chave              43 non-null int64
dtypes: float64(6), int64(3), object(6)
memory usage: 5.4+ KB


In [28]:
restantes = []

for num, row in deputados_restantes.iterrows():
    chave = str((row['chave']))
    url = 'https://dadosabertos.camara.leg.br/api/v2/deputados/' + chave
    deputado = (row['nome_urna'])
    print('deputado: ' + deputado)
    parametros = {'formato': 'json', 'itens': 100}
    resposta = requests.get(url, parametros)
    dicionario = {"deputado_id": str(resposta.json()['dados']['id']),
                      "nome_completo": resposta.json()['dados']['nomeCivil'],
                      "uri": url, 
                      "partido": resposta.json()['dados']['ultimoStatus']['siglaPartido'],
                      "legislatura": resposta.json()['dados']['ultimoStatus']['idLegislatura'],
                      "situacao": resposta.json()['dados']['ultimoStatus']['situacao'],
                      "condicao": resposta.json()['dados']['ultimoStatus']['condicaoEleitoral']}
    restantes.append(dicionario)

deputado: DILCEU SPERAFICO
deputado: MARCELO BELINATI
deputado: RONALDO FONSECA
deputado: ANDERSON FERREIRA
deputado: JARBAS
deputado: DR JOÃO
deputado: FERNANDO JORDÃO
deputado: FABIANO HORTA
deputado: WASHINGTON REIS
deputado: EDUARDO CUNHA
deputado: ANDRE FUFUCA
deputado: ZÉ CARLOS
deputado: ZÉ REINALDO
deputado: ELIZIANE GAMA
deputado: JOÃO CASTELO
deputado: MARCOS ROTTA
deputado: ATILA LINS
deputado: MORONI
deputado: ARNON BEZERRA
deputado: NELSON MARCHEZAN JUNIOR
deputado: BUSATO
deputado: MARUN
deputado: MARCIO MONTEIRO
deputado: ODELMO LEÃO
deputado: BRUNNY
deputado: PASTOR FRANKLIN LIMA
deputado: RÔMULO GOUVEIA
deputado: MANOEL JUNIOR
deputado: MOEMA GRAMACHO
deputado: PAULO MALUF
deputado: BRUNO COVAS
deputado: RENATA ABREU
deputado: EDINHO ARAUJO
deputado: DUARTE NOGUEIRA
deputado: ROBERTO GÓES
deputado: JOZI ROCHA
deputado: CESAR SOUZA
deputado: JORGE BOEIRA
deputado: MAX FILHO
deputado: GIVALDO CARIMBÃO
deputado: JHC
deputado: ALEXANDRE BALDY
deputado: JOVAIR ARANTES


In [29]:
restantes_api = pd.DataFrame(restantes)

In [30]:
restantes_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 7 columns):
condicao         43 non-null object
deputado_id      43 non-null object
legislatura      43 non-null int64
nome_completo    43 non-null object
partido          43 non-null object
situacao         43 non-null object
uri              43 non-null object
dtypes: int64(1), object(6)
memory usage: 2.4+ KB


In [45]:
restantes_api.to_csv('resultados/dep_eleitos_2014_parcial3.csv', index=False)
restantes_api.to_excel('resultados/dep_eleitos_2014_parcial3.xlsx',sheet_name='Sheet1')

## Une os arquivos para chegar nos 513 deputados

In [33]:
#deputados_eleitos_2014 = pd.read_excel('resultados/dep_eleitos_2014_unidos.xlsx',sheet_name='Sheet1', converters={'sequencial': lambda x: str(x), 'cpf': lambda x: str(x),  'deputado_id': lambda x: str(x)})

In [10]:
deputados_eleitos_2014 = pd.read_excel('resultados/dep_eleitos_2014_unidos_versaoanterior.xlsx',sheet_name='Sheet1', converters={'sequencial': lambda x: str(x), 'cpf': lambda x: str(x),  'deputado_id': lambda x: str(x)})

In [11]:
deputados_eleitos_2014.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 513 entries, 301.0 to 284.0
Data columns (total 14 columns):
uf                  513 non-null object
cargo               470 non-null object
nome_completo       513 non-null object
sequencial          513 non-null object
cpf                 513 non-null object
nome_urna           513 non-null object
partido_eleicao     513 non-null object
situacao_urna       513 non-null object
condicao            513 non-null object
deputado_id         513 non-null object
legislatura         513 non-null int64
partido_atual       513 non-null object
situacao_mandato    513 non-null object
uri                 513 non-null object
dtypes: int64(1), object(13)
memory usage: 60.1+ KB


In [12]:
deputados_eleitos_2014.reset_index().head()

,index,uf,cargo,nome_completo,sequencial,cpf,nome_urna,partido_eleicao,situacao_urna,condicao,deputado_id,legislatura,partido_atual,situacao_mandato,uri
0,301.0,RR,DEPUTADO FEDERAL,ABEL SALVADOR MESQUITA JUNIOR,230000000452,18286585291,ABEL GALINHA,PDT,ELEITO POR QP,Titular,178957,55,DEM,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...
1,79.0,PE,DEPUTADO FEDERAL,ADALBERTO CAVALCANTI RODRIGUES,170000000679,12534773453,ADALBERTO CAVALCANTI,PTB,ELEITO POR QP,Titular,178914,55,AVANTE,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...
2,318.0,BA,DEPUTADO FEDERAL,ADALBERTO SOUZA GALVAO,50000000202,21879869500,BEBETO,PSB,ELEITO POR QP,Titular,178834,55,PSB,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...
3,217.0,SE,DEPUTADO FEDERAL,ADELSON BARRETO DOS SANTOS,260000000090,23625678500,ADELSON BARRETO,PTB,ELEITO POR QP,Titular,178968,55,PR,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...
4,174.0,MT,DEPUTADO FEDERAL,ADILTON DOMINGOS SACHETTI,110000000541,45360707968,ADILTON SACHETTI,PSB,ELEITO POR QP,Titular,178903,55,PRB,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...


In [13]:
deputados_eleitos_2014.to_csv('resultados/dep_eleitos_2014_unidos.csv', index=False)

# Anota os deputados pertencentes à bancada ruralista

In [35]:
deputados_eleitos_2014 = pd.read_csv('resultados/dep_eleitos_2014_unidos.csv',sep=',',encoding = 'utf-8', converters={'sequencial': lambda x: str(x), 
                                     'cpf': lambda x: str(x), 
                                     'deputado_id': lambda x: str(x)})

In [36]:
deputados_eleitos_2014["nome_urna"] = deputados_eleitos_2014["nome_urna"].apply(f)

In [37]:
bancada = pd.read_csv("resultados/bancadaruralista_deputados.csv",sep=',',encoding = 'utf-8')

Verica nomes para checagem manual

In [38]:
nomes_naoencontrados = pd.merge(deputados_eleitos_2014, 
                        bancada, 
                        left_on='nome_urna', 
                        right_on='nome_bancada_ruralista',
                        how='outer',
                        indicator=True)

In [39]:
rdf = nomes_naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)

In [40]:
rdf.nome_bancada_ruralista

513           ANDRE AMARAL
514            CELSO JACOB
515             EDINHO BEZ
516        ELIZEU DIONIZIO
517       GUILHERME COELHO
518          MAURO PEREIRA
519        NELSON PADOVANI
520    NIVALDO ALBUQUERQUE
521     REINHOLD STEPHANES
522         RENATO ANDRADE
523       SILAS BRASILEIRO
524         XUXU DAL MOLIN
525           YEDA CRUSIUS
Name: nome_bancada_ruralista, dtype: object

In [28]:
rdf.to_csv('resultados/checar_bancada.csv', index=False)

Suplentes em 2014: <br>
ANDRE AMARAL <br>
CELSO JACOB <br>
EDINHO BEZ <br>
ELIZEU DIONIZIO <br>
GUILHERME COELHO <br>
MAURO PEREIRA <br>
NELSON PADOVANI <br>
NIVALDO ALBUQUERQUE <br>
REINHOLD STEPHANES <br>
RENATO ANDRADE <br>
SILAS BRASILEIRO <br>
XUXU DAL MOLIN <br>
YEDA CRUSIUS

In [50]:
encontrados1 = pd.merge(deputados_eleitos_2014, 
                        bancada, 
                        left_on='nome_urna', 
                        right_on='nome_bancada_ruralista')

In [51]:
encontrados1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215 entries, 0 to 214
Data columns (total 15 columns):
uf                        215 non-null object
cargo                     200 non-null object
nome_completo             215 non-null object
sequencial                215 non-null object
cpf                       215 non-null object
nome_urna                 215 non-null object
partido_eleicao           215 non-null object
situacao_urna             215 non-null object
condicao                  215 non-null object
deputado_id               215 non-null object
legislatura               215 non-null int64
partido_atual             215 non-null object
situacao_mandato          215 non-null object
uri                       215 non-null object
nome_bancada_ruralista    215 non-null object
dtypes: int64(1), object(14)
memory usage: 26.9+ KB


In [52]:
encontrados1.to_excel('resultados/eleitos_final_parte1.xlsx',sheet_name='Sheet1')

In [46]:
nomes_naoencontrados = pd.merge(deputados_eleitos_2014, 
                        bancada, 
                        left_on='nome_urna', 
                        right_on='nome_bancada_ruralista',
                        how='outer',
                        indicator=True)
ldf = nomes_naoencontrados.query("_merge == 'left_only'").drop('_merge',axis=1)

In [47]:
ldf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 298 entries, 1 to 511
Data columns (total 15 columns):
uf                        298 non-null object
cargo                     270 non-null object
nome_completo             298 non-null object
sequencial                298 non-null object
cpf                       298 non-null object
nome_urna                 298 non-null object
partido_eleicao           298 non-null object
situacao_urna             298 non-null object
condicao                  298 non-null object
deputado_id               298 non-null object
legislatura               298 non-null float64
partido_atual             298 non-null object
situacao_mandato          298 non-null object
uri                       298 non-null object
nome_bancada_ruralista    0 non-null object
dtypes: float64(1), object(14)
memory usage: 37.2+ KB


In [49]:
ldf.to_excel('resultados/eleitos_final_parte2.xlsx',sheet_name='Sheet1')

In [2]:
deputados_eleitos_2014 = pd.read_excel('resultados/eleitos_final_2014.xlsx',sheet_name='Sheet1', converters={'sequencial': lambda x: str(x), 'cpf': lambda x: str(x),  'deputado_id': lambda x: str(x)})

In [54]:
deputados_eleitos_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 15 columns):
uf                   513 non-null object
cargo                470 non-null object
nome_completo        513 non-null object
sequencial           513 non-null object
cpf                  513 non-null object
nome_urna            513 non-null object
partido_eleicao      513 non-null object
situacao_urna        513 non-null object
condicao             513 non-null object
deputado_id          513 non-null object
legislatura          513 non-null int64
partido_atual        513 non-null object
situacao_mandato     513 non-null object
uri                  513 non-null object
bancada_ruralista    513 non-null object
dtypes: int64(1), object(14)
memory usage: 60.2+ KB


# Adiciona o total de doações recebidas

In [4]:
total = pd.read_csv("resultados/total_doacoes_2014.csv",sep=',', encoding = 'utf-8', converters={'cpf_candidato': lambda x: str(x)}, decimal = ',')

In [5]:
deputados_eleitos_2014 = pd.merge(deputados_eleitos_2014, total, left_on='cpf', right_on='cpf_candidato')

In [6]:
deputados_eleitos_2014.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 0 to 512
Data columns (total 17 columns):
uf                   513 non-null object
cargo                470 non-null object
nome_completo        513 non-null object
sequencial           513 non-null object
cpf                  513 non-null object
nome_urna            513 non-null object
partido_eleicao      513 non-null object
situacao_urna        513 non-null object
condicao             513 non-null object
deputado_id          513 non-null object
legislatura          513 non-null int64
partido_atual        513 non-null object
situacao_mandato     513 non-null object
uri                  513 non-null object
bancada_ruralista    513 non-null object
cpf_candidato        513 non-null object
total_valor_doado    513 non-null float64
dtypes: float64(1), int64(1), object(15)
memory usage: 72.1+ KB


In [7]:
deputados_eleitos_2014.to_csv('resultados/dep_eleitos_2014_final.csv', index=False)

In [8]:
deputados_eleitos_2014.reset_index().head()

,index,uf,cargo,nome_completo,sequencial,cpf,nome_urna,partido_eleicao,situacao_urna,condicao,deputado_id,legislatura,partido_atual,situacao_mandato,uri,bancada_ruralista,cpf_candidato,total_valor_doado
0,0,RR,DEPUTADO FEDERAL,ABEL SALVADOR MESQUITA JUNIOR,230000000452,18286585291,ABEL GALINHA,PDT,ELEITO POR QP,Titular,178957,55,DEM,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...,sim,18286585291,199396.00
1,1,MT,DEPUTADO FEDERAL,ADILTON DOMINGOS SACHETTI,110000000541,45360707968,ADILTON SACHETTI,PSB,ELEITO POR QP,Titular,178903,55,PRB,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...,sim,45360707968,3860361.98
2,2,SP,DEPUTADO FEDERAL,ADRIANO ELI CORREA,250000001388,17610241830,ELI CORREA FILHO,DEM,ELEITO POR QP,Titular,160532,55,DEM,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...,sim,17610241830,1107579.77
3,3,MG,DEPUTADO FEDERAL,AELTON JOSE DE FREITAS,130000000672,36179140634,AELTON FREITAS,PR,ELEITO POR QP,Titular,141372,55,PR,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...,sim,36179140634,1817657.03
4,4,RS,DEPUTADO FEDERAL,AFONSO ANTUNES DA MOTTA,210000000224,10777296004,AFONSO MOTTA,PDT,ELEITO POR MÉDIA,Titular,178835,55,PDT,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...,sim,10777296004,1354345.25


Faz um merge dos dados de TE com a lista acima para pegar todas as linhas que têm os documentos

In [9]:
deputados_eleitos_2014.to_excel('resultados/eleitos_final_2014.xlsx',sheet_name='Sheet1')

In [2]:
deputados_eleitos_2014 = pd.read_excel('resultados/eleitos_final_2014.xlsx',sheet_name='Sheet1', converters={'politico_cpf': lambda x: str(x)}, decimal = ',')

In [3]:
deputados_eleitos_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 10 columns):
politico_estado                     513 non-null object
politico_nome                       513 non-null object
politico_cpf                        513 non-null object
politico_nome_urna                  513 non-null object
politico_partido_eleicao            513 non-null object
politico_partido_atual              513 non-null object
politico_bancada_ruralista          513 non-null object
politico_total_recebido             513 non-null float64
politico_cargo_bancada_ruralista    14 non-null object
politico_candidato_cargo            0 non-null float64
dtypes: float64(2), object(8)
memory usage: 40.2+ KB


Acessa os candidatos 2018

In [4]:
candidatos_2018 = pd.read_excel('resultados/candidatos_2018.xlsx',sheet_name='Sheet1', converters={'sequencial': lambda x: str(x), 
                                                                                              'cpf': lambda x: str(x),  
                                                                                              'numero_urna': lambda x: str(x)})

In [5]:
candidatos_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27887 entries, 0 to 414
Data columns (total 4 columns):
uf_eleicao_disputa    27887 non-null object
cargo_em_disputa      27887 non-null object
nome_urna             27887 non-null object
cpf                   27887 non-null object
dtypes: object(4)
memory usage: 1.1+ MB


In [6]:
deputados_eleitos_2014_parcial = pd.merge(deputados_eleitos_2014, candidatos_2018, left_on='politico_cpf', right_on='cpf')

In [7]:
deputados_eleitos_2014_parcial.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466 entries, 0 to 465
Data columns (total 14 columns):
politico_estado                     466 non-null object
politico_nome                       466 non-null object
politico_cpf                        466 non-null object
politico_nome_urna                  466 non-null object
politico_partido_eleicao            466 non-null object
politico_partido_atual              466 non-null object
politico_bancada_ruralista          466 non-null object
politico_total_recebido             466 non-null float64
politico_cargo_bancada_ruralista    12 non-null object
politico_candidato_cargo            0 non-null float64
uf_eleicao_disputa                  466 non-null object
cargo_em_disputa                    466 non-null object
nome_urna                           466 non-null object
cpf                                 466 non-null object
dtypes: float64(2), object(12)
memory usage: 54.6+ KB


In [8]:
deputados_eleitos_2014_parcial.to_excel('resultados/eleitos_final_2014_com_cargos.xlsx',sheet_name='Sheet1')

In [63]:
nomes_naoencontrados = pd.merge(deputados_eleitos_2014, 
                        candidatos_2018, 
                        left_on='politico_cpf', 
                        right_on='cpf',
                        how='outer',
                        indicator=True)

ldf = nomes_naoencontrados.query("_merge == 'left_only'").drop('_merge',axis=1)

ldf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 8 to 499
Data columns (total 14 columns):
politico_estado                     50 non-null object
politico_nome                       50 non-null object
politico_cpf                        50 non-null object
politico_nome_urna                  50 non-null object
politico_partido_eleicao            50 non-null object
politico_partido_atual              50 non-null object
politico_bancada_ruralista          50 non-null object
politico_total_recebido             50 non-null float64
politico_cargo_bancada_ruralista    2 non-null object
politico_candidato_cargo            0 non-null float64
uf_eleicao_disputa                  0 non-null object
cargo_em_disputa                    0 non-null object
nome_urna                           0 non-null object
cpf                                 0 non-null object
dtypes: float64(2), object(12)
memory usage: 5.9+ KB


In [65]:
ldf.to_excel('resultados/eleitos_final_2014_sem_cargos.xlsx',sheet_name='Sheet1')